# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1024 07:34:34.188473    6769 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1024 07:34:49.921603    6801 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Mon Oct 24 07:34:52 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [338]:
# HOMEWORKS
# EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_4_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/distributed/fashionmnist_fashionmnistcnn/epoch_25/experiment_10e-4_25.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [339]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1024 16:47:59.441731   22277 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1024 16:48:02.775482   22294 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Mon Oct 24 16:48:04 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [340]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1024 16:48:10.968725   22314 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-24-2022 14:48:11 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-24-2022 14:48:11 root         INFO     Starting in cluster mode.
10-24-2022 14:48:11 root         INFO     Starting with experiment replication: 0 with seed: 42
10-24-2022 14:48:11 root         INFO     Starting as Orchestrator
10-24-2022 14:48:11 root         INFO     Starting Orchestrator, initializing resources....
10-24-2022 14:48:11 root         INFO     Loading in cluster configuration file
10-24-2022 14:48:11 root         INFO     Pointing configuration to in cluster configuration.
10-24-2022 14:48:11 root         INFO     Starting cluster manager
10-24-2022 14:48:11 ClusterManager INFO     Spinning up cluster manag

In [368]:
CURRENT_POD_ID="3cf8661b-821a-44cb-92da-7770af6f34eb"
# To get logs from learners (example)
kubectl logs -n test trainjob-$CURRENT_POD_ID-master-0

W1024 17:57:16.269022   27129 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-24-2022 14:48:20 root         INFO     Loading file config/configuration.fltk.json
10-24-2022 14:48:20 root         INFO     Starting in client mode
10-24-2022 14:48:20 root         INFO     Starting with host=trainjob-3cf8661b-821a-44cb-92da-7770af6f34eb-master-0 and port=23456
10-24-2022 14:48:20 root         INFO     Initializing backend for training process: gloo
10-24-2022 15:32:48 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-24-2022 15:32:48 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-24-2022 15:32:48 root         INFO     Starting Creating client with 0
10-24-2022 15:32:48 Client-0-3

10-24-2022 15:55:35 Client-0-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [24,    50] loss: 2.227
10-24-2022 15:55:58 Client-0-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [24,   100] loss: 2.229
10-24-2022 15:56:10 Client-0-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,     0] loss: 0.046
10-24-2022 15:56:33 Client-0-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,    50] loss: 2.223
10-24-2022 15:56:56 Client-0-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,   100] loss: 2.224
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=63189, duration_test=4080, loss_train=2.336482033729553, accuracy=9.8, loss=46.49000310897827, class_precision=array([0.09831461, 0.06441718, 0.        , 0.        , 0.        ,
       0.10547945, 0.        , 0.04225352, 0.19811321, 0.25      ]), class_recall=array([0.25925926, 0.15671642, 0.        , 0.        , 0.        ,
       0.33478261, 0.        , 0.01229508, 0.1779661 , 0.0464135 ]), confusion_mat=array([[ 70,

       0.34347826, 0.00452489, 0.04098361, 0.19491525, 0.08860759]), confusion_mat=array([[ 75, 123,   5,   0,   0,  39,   0,   1,  24,   3],
       [  6,  52,   0,   0,   0, 206,   0,   0,   4,   0],
       [ 91, 120,   0,   0,   1,  29,   0,   7,  10,  11],
       [ 15,  87,   4,   1,   0, 131,   0,   2,  28,   2],
       [ 47, 160,   2,   0,   1,  18,   0,   5,  14,   8],
       [ 84,   6,   0,  28,   2,  79,   1,   3,  17,  10],
       [ 77,  97,   0,   1,   0,  20,   1,   9,   9,   7],
       [100,   0,   0,  17,  14,  31,   0,  10,  70,   2],
       [ 83,   5,   4,   1,   3,  35,   0,  57,  46,   2],
       [ 87,   5,   3,   3,   3,  91,   2,   7,  15,  21]])), EpochData(epoch_id=9, num_epochs=26, duration_train=523891, duration_test=3984, loss_train=2.29633713722229, accuracy=11.72, loss=45.69264602661133, class_precision=array([0.11485452, 0.08345979, 0.        , 0.01851852, 0.11111111,
       0.11844078, 0.2       , 0.10679612, 0.19183673, 0.31343284]), class_recall=array([0.2

       0.13687601, 0.16666667, 0.11965812, 0.19685039, 0.4       ]), class_recall=array([0.31481481, 0.27985075, 0.        , 0.00740741, 0.02745098,
       0.36956522, 0.00452489, 0.05737705, 0.21186441, 0.15189873]), confusion_mat=array([[ 85, 119,   5,   0,   0,  31,   0,   1,  25,   4],
       [  5,  75,   1,   0,   0, 182,   0,   1,   4,   0],
       [ 90, 115,   0,   0,   2,  26,   0,  11,  11,  14],
       [ 13,  95,   7,   2,   1, 116,   0,   2,  32,   2],
       [ 43, 154,   4,   0,   7,  17,   0,   5,  16,   9],
       [ 69,   5,   0,  30,   5,  85,   2,   2,  19,  13],
       [ 81,  94,   2,   1,   0,  17,   1,   9,   9,   7],
       [ 76,   0,   0,  28,  18,  32,   0,  14,  74,   2],
       [ 74,   4,   4,   1,   4,  32,   0,  64,  50,   3],
       [ 78,   4,   4,   4,   3,  83,   3,   8,  14,  36]])), EpochData(epoch_id=17, num_epochs=26, duration_train=990681, duration_test=3894, loss_train=2.2591362142562867, accuracy=14.56, loss=44.95288157463074, class_precision=array([

       0.15853659, 0.09090909, 0.13740458, 0.1978022 , 0.3853211 ]), class_recall=array([0.33333333, 0.3619403 , 0.00743494, 0.01111111, 0.05882353,
       0.39565217, 0.00452489, 0.07377049, 0.22881356, 0.17721519]), confusion_mat=array([[ 90, 117,   6,   0,   0,  26,   0,   2,  25,   4],
       [  6,  97,   1,   0,   0, 160,   0,   1,   3,   0],
       [ 92, 102,   2,   0,   6,  22,   0,  12,  14,  19],
       [ 10, 104,  11,   3,   1, 100,   0,   2,  37,   2],
       [ 41, 140,   6,   0,  15,  16,   1,   6,  19,  11],
       [ 58,   5,   0,  30,   4,  91,   4,   3,  19,  16],
       [ 82,  88,   2,   1,   3,  17,   1,   9,  10,   8],
       [ 57,   0,   1,  39,  19,  31,   0,  18,  77,   2],
       [ 63,   3,   6,   1,   5,  28,   1,  70,  54,   5],
       [ 71,   3,   4,   4,   3,  83,   4,   8,  15,  42]])), EpochData(epoch_id=25, num_epochs=26, duration_train=1455267, duration_test=4101, loss_train=2.224372873306274, accuracy=17.04, loss=44.26101493835449, class_precision=array([

In [369]:
# To get logs from learners (federated learning)
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-0

W1024 17:57:52.383921   27212 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-24-2022 14:48:25 root         INFO     Loading file config/configuration.fltk.json
10-24-2022 14:48:25 root         INFO     Starting in client mode
10-24-2022 14:48:26 root         INFO     Starting with host=trainjob-3cf8661b-821a-44cb-92da-7770af6f34eb-master-0 and port=23456
10-24-2022 14:48:26 root         INFO     Initializing backend for training process: gloo
10-24-2022 15:32:48 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-24-2022 15:32:48 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-24-2022 15:32:48 root         INFO

10-24-2022 15:55:12 Client-1-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [24,     0] loss: 0.046
10-24-2022 15:55:35 Client-1-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [24,    50] loss: 2.227
10-24-2022 15:55:58 Client-1-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [24,   100] loss: 2.229
10-24-2022 15:56:10 Client-1-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,     0] loss: 0.046
10-24-2022 15:56:33 Client-1-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,    50] loss: 2.223
10-24-2022 15:56:56 Client-1-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,   100] loss: 2.224
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=63202, duration_test=4091, loss_train=2.336482033729553, accuracy=8.48, loss=46.64841151237488, class_precision=array([0.06824513, 0.0830721 , 0.        , 0.        , 0.04      ,
       0.09673025, 0.        , 0.        , 0.14150943, 0.18181818]), class_recall=array([0.20588235, 0.19702602, 0.        , 0.        , 0.00434

       0.11079137, 0.        , 0.03389831, 0.13636364, 0.33846154]), class_recall=array([0.23109244, 0.24163569, 0.        , 0.        , 0.00869565,
       0.28731343, 0.        , 0.00784314, 0.13983051, 0.08661417]), confusion_mat=array([[ 55, 110,   1,   1,   0,  43,   0,   2,  23,   3],
       [  5,  65,   0,   1,   0, 191,   0,   1,   5,   1],
       [ 72, 118,   0,   2,   1,  23,   0,   5,   9,   9],
       [ 11,  65,   4,   0,   0, 137,   0,   1,  36,   3],
       [ 51, 138,   1,   0,   2,  18,   0,   1,  13,   6],
       [ 91,   9,   0,  49,   0,  77,   6,   3,  23,  10],
       [ 76, 129,   1,   0,   1,  30,   0,   5,   6,   6],
       [103,   0,   0,  26,  19,  30,   0,   2,  73,   2],
       [107,   8,   1,   1,   6,  38,   1,  38,  33,   3],
       [ 85,   6,   3,   2,   4, 108,   2,   1,  21,  22]])), EpochData(epoch_id=9, num_epochs=26, duration_train=523905, duration_test=3900, loss_train=2.29633713722229, accuracy=10.6, loss=45.856430768966675, class_precision=array([0.0

       0.12732919, 0.        , 0.15116279, 0.15      , 0.41463415]), class_recall=array([0.25630252, 0.30855019, 0.0125523 , 0.        , 0.02608696,
       0.30597015, 0.        , 0.05098039, 0.16525424, 0.13385827]), confusion_mat=array([[ 61, 107,   1,   2,   0,  37,   0,   3,  25,   2],
       [  5,  83,   0,   1,   0, 172,   0,   1,   6,   1],
       [ 71, 117,   3,   2,   2,  17,   0,   6,  10,  11],
       [  9,  72,   6,   0,   0, 126,   0,   1,  40,   3],
       [ 50, 133,   1,   0,   6,  15,   1,   5,  13,   6],
       [ 80,   7,   0,  52,   0,  82,   6,   3,  25,  13],
       [ 79, 127,   1,   0,   1,  29,   0,   5,   6,   6],
       [ 85,   0,   0,  36,  19,  29,   0,  13,  71,   2],
       [ 96,   6,   2,   1,   6,  34,   1,  47,  39,   4],
       [ 74,   3,   4,   2,   4, 103,   3,   2,  25,  34]])), EpochData(epoch_id=17, num_epochs=26, duration_train=990695, duration_test=4109, loss_train=2.2591362142562867, accuracy=12.92, loss=45.11964130401611, class_precision=array([

       0.14691152, 0.05555556, 0.21698113, 0.15942029, 0.42608696]), class_recall=array([0.28151261, 0.34200743, 0.0209205 , 0.00389105, 0.04782609,
       0.32835821, 0.00393701, 0.09019608, 0.18644068, 0.19291339]), confusion_mat=array([[ 67, 109,   1,   2,   1,  25,   1,   3,  27,   2],
       [  4,  92,   0,   1,   0, 163,   0,   1,   7,   1],
       [ 71, 107,   5,   4,   3,  15,   1,   7,  11,  15],
       [  8,  75,   8,   1,   0, 117,   1,   2,  40,   5],
       [ 51, 122,   2,   0,  11,  13,   1,   6,  16,   8],
       [ 64,   4,   0,  59,   0,  88,   7,   4,  26,  16],
       [ 77, 120,   4,   0,   5,  26,   1,   4,   8,   9],
       [ 65,   0,   0,  44,  21,  28,   0,  23,  70,   4],
       [ 83,   6,   3,   1,   8,  31,   2,  52,  44,   6],
       [ 63,   1,   7,   3,   3,  93,   4,   4,  27,  49]])), EpochData(epoch_id=25, num_epochs=26, duration_train=1455281, duration_test=4023, loss_train=2.224372873306274, accuracy=15.64, loss=44.42855644226074, class_precision=array([

In [370]:
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-1

W1024 17:58:08.152080   27248 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-24-2022 15:32:44 root         INFO     Loading file config/configuration.fltk.json
10-24-2022 15:32:44 root         INFO     Starting in client mode
10-24-2022 15:32:44 root         INFO     Starting with host=trainjob-3cf8661b-821a-44cb-92da-7770af6f34eb-master-0 and port=23456
10-24-2022 15:32:44 root         INFO     Initializing backend for training process: gloo
10-24-2022 15:32:48 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 2
10-24-2022 15:32:48 torch.distributed.distributed_c10d INFO     Rank 2: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-24-2022 15:32:48 root         INFO

10-24-2022 15:55:12 Client-2-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [24,     0] loss: 0.046
10-24-2022 15:55:35 Client-2-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [24,    50] loss: 2.227
10-24-2022 15:55:58 Client-2-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [24,   100] loss: 2.229
10-24-2022 15:56:10 Client-2-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,     0] loss: 0.046
10-24-2022 15:56:33 Client-2-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,    50] loss: 2.223
10-24-2022 15:56:56 Client-2-3cf8661b-821a-44cb-92da-7770af6f34eb INFO     [25,   100] loss: 2.224
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=63203, duration_test=4091, loss_train=2.336482033729553, accuracy=9.76, loss=46.779216289520264, class_precision=array([0.07746479, 0.05640244, 0.        , 0.        , 0.08      ,
       0.10571429, 0.        , 0.08695652, 0.27522936, 0.21276596]), class_recall=array([0.22540984, 0.16086957, 0.        , 0.        , 0.0076

       0.10752688, 0.        , 0.07954545, 0.2780083 , 0.32394366]), class_recall=array([0.24180328, 0.19565217, 0.00854701, 0.        , 0.01915709,
       0.29661017, 0.        , 0.02755906, 0.24723247, 0.09236948]), confusion_mat=array([[ 59, 117,   3,   0,   0,  40,   0,   1,  23,   1],
       [  4,  45,   0,   0,   0, 176,   0,   0,   5,   0],
       [ 80, 106,   2,   1,   4,  22,   0,   8,   6,   5],
       [ 15,  74,   6,   0,   0, 129,   0,   2,  24,   2],
       [ 45, 154,   2,   1,   5,  20,   0,   6,  15,  13],
       [ 83,   4,   0,  47,   3,  70,   1,   1,  13,  14],
       [ 80, 130,   3,   2,   0,  29,   0,   7,   9,   9],
       [103,   1,   0,  28,  17,  24,   0,   7,  73,   1],
       [ 93,  12,   1,   1,   4,  39,   0,  51,  67,   3],
       [101,   3,   3,   2,   2, 102,   2,   5,   6,  23]])), EpochData(epoch_id=9, num_epochs=26, duration_train=523905, duration_test=3899, loss_train=2.29633713722229, accuracy=11.32, loss=45.966609716415405, class_precision=array([0.

       0.12335526, 0.        , 0.1025641 , 0.26819923, 0.3956044 ]), class_recall=array([0.26639344, 0.23913043, 0.01282051, 0.00396825, 0.03831418,
       0.31779661, 0.        , 0.04724409, 0.25830258, 0.14457831]), confusion_mat=array([[ 65, 113,   4,   0,   0,  33,   0,   1,  27,   1],
       [  4,  55,   1,   1,   0, 164,   0,   0,   5,   0],
       [ 80,  99,   3,   1,   5,  20,   0,  12,   6,   8],
       [ 13,  83,   6,   1,   1, 116,   0,   2,  27,   3],
       [ 45, 150,   4,   1,  10,  17,   0,   5,  17,  12],
       [ 69,   4,   0,  51,   3,  75,   1,   5,  13,  15],
       [ 81, 128,   3,   2,   3,  24,   0,  10,   9,   9],
       [ 83,   1,   0,  32,  18,  27,   0,  12,  80,   1],
       [ 78,   8,   1,   1,   5,  36,   2,  64,  70,   6],
       [ 87,   2,   4,   7,   2,  96,   2,   6,   7,  36]])), EpochData(epoch_id=17, num_epochs=26, duration_train=990696, duration_test=4003, loss_train=2.2591362142562867, accuracy=13.4, loss=45.214303731918335, class_precision=array([

       0.13513514, 0.        , 0.16083916, 0.27573529, 0.45299145]), class_recall=array([0.29508197, 0.30869565, 0.01282051, 0.00793651, 0.06896552,
       0.31779661, 0.        , 0.09055118, 0.27675277, 0.21285141]), confusion_mat=array([[ 72, 116,   3,   2,   0,  21,   0,   2,  27,   1],
       [  4,  71,   1,   2,   0, 146,   0,   0,   6,   0],
       [ 79,  92,   3,   2,   9,  18,   0,  15,   7,   9],
       [ 15,  87,   7,   2,   1, 107,   0,   2,  28,   3],
       [ 45, 143,   5,   1,  18,  14,   0,   5,  18,  12],
       [ 58,   4,   0,  55,   3,  75,   2,   5,  13,  21],
       [ 80, 124,   5,   3,   5,  23,   0,  11,   9,   9],
       [ 64,   1,   0,  35,  17,  30,   0,  23,  82,   2],
       [ 66,   6,   1,   1,   6,  34,   3,  72,  75,   7],
       [ 75,   1,   4,   7,   4,  87,   3,   8,   7,  53]])), EpochData(epoch_id=25, num_epochs=26, duration_train=1455281, duration_test=4018, loss_train=2.224372873306274, accuracy=16.16, loss=44.51207900047302, class_precision=array([

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [371]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1024 17:58:55.030798   27313 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1024 17:58:55.706550   27328 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [372]:
helm uninstall -n test experiment-orchestrator

W1024 17:59:14.695643   27365 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [373]:
helm uninstall extractor -n test

W1024 17:59:25.034109   27380 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [374]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1024 18:00:44.254520   27505 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-0c3fe856-c229-4249-a8fd-d84f214cc667" deleted
pytorchjob.kubeflow.org "trainjob-16208433-5b96-434c-90dd-947c0196c49b" deleted
pytorchjob.kubeflow.org "trainjob-176d67a9-c8ff-4e00-b918-48b61b1fb406" deleted
pytorchjob.kubeflow.org "trainjob-193dbe4a-a720-4b51-870b-c40edfbfa9ed" deleted
pytorchjob.kubeflow.org "trainjob-251987fe-d5b4-444b-ae0c-5ca07bd7276b" deleted
pytorchjob.kubeflow.org "trainjob-25b6702b-2e3c-4ce0-b80e-a3d39cda0b26" deleted
pytorchjob.kubeflow.org "trainjob-33e5f763-e90b-4218-b9f8-2315a4cc6da9" deleted
pytorchjob.kubeflow.org "trainjob-3447eb44-b1ba-4b85-9ff1-d0be035c79f4" deleted
pytorchjob.kubeflow.org "trainjob-3a9d09c8-a62d-496f-9cce-711c75ddd051" deleted
pytorchjob.kubeflow.org "trai